In [2]:

import duckdb 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from src.feature_engineering import feature_engineering_lag , feature_engineering_delta


%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb

In [3]:
data_path = "data/"
data_file = "competencia_01.csv"
df= pd.read_csv(data_path + data_file)

C:\Users\chris\AppData\Local\Temp\ipykernel_13436\4193464990.py:3: DtypeWarning: Columns (154) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(data_path + data_file)


#

In [4]:
cat_cols =[]
num_cols=[]
for c in df.columns:
    if (df[c].nunique() <= 5):
        cat_cols.append(c)
    else:
        num_cols.append(c)

In [5]:
lista_t=[c for c in list(map(lambda x : x if x[0]=='t' and x not in cat_cols else np.nan ,df.columns )) if pd.notna(c)]
lista_c=[c for c in list(map(lambda x : x if x[0]=='c' and x not in cat_cols else np.nan ,df.columns )) if pd.notna(c)]
lista_m=[c for c in list(map(lambda x : x if x[0]=='m' and x not in cat_cols else np.nan ,df.columns )) if pd.notna(c)]
lista_r=[c for c in df.columns if c not in (lista_t + lista_c + lista_m + cat_cols)]
print(len(lista_t) , len(lista_c) , len(lista_m) , len(lista_r) , len(cat_cols))
print(len(lista_t) + len(lista_c) + len(lista_m) +len(lista_r) + len(cat_cols))

0 48 45 42 20
155


In [6]:
col_drops=["cliente_edad","numero_de_cliente","cliente_antiguedad"]

In [7]:
cols_transf_previo=lista_m + lista_c+ lista_r
cols_trans = [c for c in cols_transf_previo if c not in col_drops]
cols_trans

['mrentabilidad',
 'mrentabilidad_annual',
 'mcomisiones',
 'mactivos_margen',
 'mpasivos_margen',
 'mcuenta_corriente_adicional',
 'mcuenta_corriente',
 'mcaja_ahorro',
 'mcaja_ahorro_adicional',
 'mcaja_ahorro_dolares',
 'mcuentas_saldo',
 'mautoservicio',
 'mtarjeta_visa_consumo',
 'mtarjeta_master_consumo',
 'mprestamos_personales',
 'mprestamos_prendarios',
 'mprestamos_hipotecarios',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'minversion1_pesos',
 'minversion1_dolares',
 'minversion2',
 'mpayroll',
 'mpayroll2',
 'mcuenta_debitos_automaticos',
 'mttarjeta_visa_debitos_automaticos',
 'mttarjeta_master_debitos_automaticos',
 'mpagodeservicios',
 'mpagomiscuentas',
 'mcajeros_propios_descuentos',
 'mtarjeta_visa_descuentos',
 'mtarjeta_master_descuentos',
 'mcomisiones_mantenimiento',
 'mcomisiones_otras',
 'mforex_buy',
 'mforex_sell',
 'mtransferencias_recibidas',
 'mtransferencias_emitidas',
 'mextraccion_autoservicio',
 'mcheques_depositados',
 'mcheques_emitidos',
 'mcheque

In [8]:
df_2 = feature_engineering_lag(df ,cols_trans , 2)
df_3 = feature_engineering_delta(df_2 ,cols_trans , 2)

ejecucion lag finalizada
ejecucion delta finalizada


In [9]:
%%sql 
SELECT numero_de_cliente,foto_mes,mrentabilidad_annual,mrentabilidad_annual_lag_1 , mrentabilidad_annual_lag_2 , delta_1_mrentabilidad_annual , delta_2_mrentabilidad_annual
FROM df_3
ORDER BY (numero_de_cliente,foto_mes )

,numero_de_cliente,foto_mes,mrentabilidad_annual,mrentabilidad_annual_lag_1,mrentabilidad_annual_lag_2,delta_1_mrentabilidad_annual,delta_2_mrentabilidad_annual
0,249221323,202101,15691.10,NaN,NaN,NaN,NaN
1,249221323,202102,19103.66,15691.10,NaN,3412.56,NaN
2,249221323,202103,24020.21,19103.66,15691.10,4916.55,8329.11
3,249221323,202104,29189.81,24020.21,19103.66,5169.60,10086.15
4,249221323,202105,31505.78,29189.81,24020.21,2315.97,7485.57
...,...,...,...,...,...,...,...
978434,1598368433,202106,4123.09,NaN,NaN,NaN,NaN
978435,1598419415,202106,-2.26,NaN,NaN,NaN,NaN
978436,1598425905,202106,5.92,NaN,NaN,NaN,NaN
978437,1598470389,202106,244.96,NaN,NaN,NaN,NaN


In [10]:
%%sql 
SELECT numero_de_cliente,foto_mes,mrentabilidad_annual,mrentabilidad_annual_lag_1 , mrentabilidad_annual_lag_2 , delta_1_mrentabilidad_annual , delta_2_mrentabilidad_annual
FROM df_3
ORDER BY (numero_de_cliente,foto_mes )

,numero_de_cliente,foto_mes,mrentabilidad_annual,mrentabilidad_annual_lag_1,mrentabilidad_annual_lag_2,delta_1_mrentabilidad_annual,delta_2_mrentabilidad_annual
0,249221323,202101,15691.10,NaN,NaN,NaN,NaN
1,249221323,202102,19103.66,15691.10,NaN,3412.56,NaN
2,249221323,202103,24020.21,19103.66,15691.10,4916.55,8329.11
3,249221323,202104,29189.81,24020.21,19103.66,5169.60,10086.15
4,249221323,202105,31505.78,29189.81,24020.21,2315.97,7485.57
...,...,...,...,...,...,...,...
978434,1598368433,202106,4123.09,NaN,NaN,NaN,NaN
978435,1598419415,202106,-2.26,NaN,NaN,NaN,NaN
978436,1598425905,202106,5.92,NaN,NaN,NaN,NaN
978437,1598470389,202106,244.96,NaN,NaN,NaN,NaN


In [11]:
df_3

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,delta_1_Visa_fechaalta,delta_2_Visa_fechaalta,delta_1_Visa_mconsumototal,delta_2_Visa_mconsumototal,delta_1_Visa_cconsumos,delta_2_Visa_cconsumos,delta_1_Visa_cadelantosefectivo,delta_2_Visa_cadelantosefectivo,delta_1_Visa_mpagominimo,delta_2_Visa_mpagominimo
0,302071279,202101,1,0,0,48,373,1853.73,35889.51,1197.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,302071279,202102,1,0,0,48,374,2405.86,35762.22,1399.90,...,28.0,NaN,-1642.28,NaN,2.0,NaN,0.0,NaN,-140.76,NaN
2,302071279,202103,1,0,0,48,375,1677.39,33914.66,1831.55,...,31.0,59.0,-5270.46,-6912.74,-1.0,1.0,0.0,0.0,-281.52,-422.28
3,302071279,202104,1,0,0,48,376,1030.32,31535.20,1322.56,...,30.0,61.0,3212.05,-2058.41,1.0,0.0,0.0,0.0,175.95,-105.57
4,302071279,202105,1,0,0,48,377,-184.31,28082.58,1109.39,...,31.0,61.0,-4277.86,-1065.81,-1.0,0.0,0.0,0.0,-1560.09,-1384.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,1596521562,202106,1,0,0,18,1,38.96,38.96,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978435,1596723860,202106,0,0,0,58,1,5.87,5.87,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978436,1597542829,202106,1,0,0,21,1,0.19,0.19,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978437,1597771164,202106,0,0,0,27,1,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
mes =202104
f = df_3["foto_mes"] == mes
df = df_3.loc[f]

X_train=df.drop(columns="clase_ternaria")
y_train = df["clase_ternaria"].copy()

In [13]:
y_train

3         Continua
9         Continua
15        Continua
21        Continua
27        Continua
            ...   
978403    Continua
978407    Continua
978412    Continua
978415    Continua
978422    Continua
Name: clase_ternaria, Length: 163418, dtype: object

In [16]:
df

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,delta_1_Visa_fechaalta,delta_2_Visa_fechaalta,delta_1_Visa_mconsumototal,delta_2_Visa_mconsumototal,delta_1_Visa_cconsumos,delta_2_Visa_cconsumos,delta_1_Visa_cadelantosefectivo,delta_2_Visa_cadelantosefectivo,delta_1_Visa_mpagominimo,delta_2_Visa_mpagominimo
3,302071279,202104,1,0,0,48,376,1030.32,31535.20,1322.56,...,30.0,61.0,3212.05,-2058.41,1.0,0.0,0.0,0.0,175.95,-105.57
9,302089801,202104,1,0,0,61,280,2056.29,20175.41,529.01,...,30.0,61.0,0.00,-385.91,0.0,-1.0,0.0,0.0,-58.65,117.30
15,302105993,202104,1,0,0,62,323,9882.24,118366.76,4066.80,...,30.0,61.0,-3023.14,-2691.46,-2.0,-4.0,0.0,0.0,-1114.35,-1419.33
21,302191168,202104,1,0,0,74,323,1027.63,2323.14,1654.22,...,30.0,61.0,3167.35,351.66,-5.0,-10.0,0.0,0.0,234.60,-117.30
27,302202987,202104,1,0,0,58,7,-1205.57,1240.97,814.66,...,30.0,61.0,2705.45,-10187.71,0.0,-2.0,0.0,0.0,-1184.73,-1536.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978403,1587996892,202104,1,0,0,50,2,1418.48,1276.30,1120.36,...,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290.30,NaN
978407,1589236724,202104,1,0,0,32,2,9.93,9.93,0.00,...,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
978412,1590035787,202104,1,0,0,29,1,-504.15,-504.15,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978415,1590285711,202104,1,0,0,30,1,212.60,212.60,4.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
type(y_train)

pandas.core.series.Series

In [15]:
type(X_train)

pandas.core.frame.DataFrame